### Redoing pangenome calculations using Roary and Sourmash.
Sourmash pangenome plugin was changed a bit. 
Pipeline collects all microbial genomes of species. Then dereplicates them at 99% ANI, and runs prokka for protein predictions. The protein predictions are used by roary, to define core, soft core, shell and cloud genes for the microbial species. 

Roary returns a (not-so-handy) presence/absence file for all genes found in all strains. This needs to be input in a python script to get lists of gene names, that are in each category. Can use that to filter fasta files, to get a 'cloud' and 'core' fasta file in DNA space. This is then (again) put into prokka, to predict the genes, so that we also have a 'cloud' and 'core' fasta in AA space.

For now, only uses reference strains. Will add using host (pig) specific strains later.


In [ ]:
# update plugin
mamba activate pangenomics_dev

# upgrade
pip install --upgrade --force-reinstall git+https://github.com/dib-lab/sourmash_plugin_pangenomics.git

# install tables plugin
mamba activate pangenomics_dev
git clone https://github.com/sourmash-bio/sourmash_plugin_tables.git
cd sourmash_plugin_tables
pip install -e .


#### Steps to get from a species name to a pangenome sketch (protein and nucleotide)
I want to be able to go from species name, to a merged pangenome sketch, in both protein and nucleotide sketch. For now i am only interested in the core and cloud sketch, not in soft core or shell. 

I would like to add a separate workflow in which we can add species specific strains, such as all newly assembled pig stuff, but would first like for it to work on just reference genomes. 

**Steps**:
1. get all genomes of species of interest, then dereplicate (1_collect_genomes.smk)
2. Run prokka on the genomes, then run roary (2_roary.smk)
3. Take roary output, and make separate fasta files for core and cloud genes (3_genes_to_hash.smk). 

**TODO**: Translate to protein and convert into hashes (3_genes_to_hash.smk), so that we have a sketch for all core and all cloud genes

In [ ]:
# where is the tax file:
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250411_mag_taxonomy.tsv

### Steps to add: 
Not yet implemented:
- nucl fasta to sig in dna space

Either:
- nucl fasta translated to protein
OR
- nucl fasta into prokka, then faa file to protein

Need to compare outputs first

In [ ]:
# Do a test with just reference genomes. (Megasphaera elsdenii)
mkdir test_pipeline

# see Snakefile
# how to go from 2 lists of species to a folder with all of them: For drep. 
srun --account=ctbrowngrp -p med2 -J roary -t 4:00:00 -c 24 --mem=80gb --pty bash
mamba activate branchwater-skipmer

snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 24 --rerun-incomplete -k -n

In [ ]:
# gene presence absemce file
# how to get to those lists, then files?
python /group/ctbrowngrp2/amhorst/2025-pangenome/workflow/scripts/count_genes.py \
gene_presence_absence.Rtab test.csv

# gives files hard_core.txt, soft_core.txt, cloud.txt, shell.txt
# with group numbers. Now how to get the sequence?
# from pan_genome_reference.fa --> proteins?
# and easiest is probs converting these sequences to AA
mamba activate seqkit

# test grabbing seqs of interest
seqkit grep -r -n -f /group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/roary/hard_core.txt \
/group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/roary/pan_genome_reference.fa \
-o /group/ctbrowngrp2/amhorst/2025-pangenome/results/test_pipeline/m_elsdenii/sourmash/m_elsdenii.core.fa



In [ ]:
# try it out in protein space:
# downloaf and sketch metagenomes in translate
cp m_elsdenii/sourmash/m_elsdenii.cloud.fa ./protein_space/

# translate to protein with prokka
mamba activate prokka

prokka --kingdom Bacteria --outdir cloud_genes \
--norrna --notrna --prefix cloud --force \
--locustag cloud m_elsdenii.cloud.fa 

prokka --kingdom Bacteria --outdir core_genes \
--norrna --notrna --prefix core --force \
--locustag core m_elsdenii.core.fa 


# we can prob do sketch translate, then dont have to go back to prokka
# lets compare outputs
sourmash sketch translate -p k=10,scaled=500 m_elsdenii.core.fa -o m_elsdenii.core.tr.k10.zip
sourmash sketch translate -p k=10,scaled=500 m_elsdenii.cloud.fa -o m_elsdenii.cloud.tr.k10.zip

# also do protein sketches and see if different when crossing with metaG
sourmash sketch protein cloud_genes/cloud.faa -p k=10,scaled=500 -o cloud_genes/cloud.prot.zip
sourmash sketch protein core_genes/core.faa -p k=10,scaled=500 -o core_genes/core.prot.zip

### TODO:
Adding non-reference genomes. (now total of 95, 28 pig specific)

In [ ]:
# add a drep rule for only the reference genomes and one for all
srun --account=ctbrowngrp -p med2 -J roary -t 2:00:00 -c 24 --mem=50gb --pty bash
mamba activate branchwater-skipmer

dRep dereplicate drep_allMAGs -p 24 \
--ignoreGenomeQuality -pa 0.9 -sa 0.99 -nc 0.30 -cm larger \
-g MAGs/*.fasta 


In [ ]:
# now for protein space:
# translated to protein with prokka

sourmash sketch translate -p k=10,scaled=500

sourmash sketch protein -p k=10,scaled=500 genome.faa

sourmash sketch dna m_elsdenii.cloud.fa -o m_elsdenii.cloud.k21.zip -p k=21,scaled=1000 

# try pangenome merge and ranktable (m2n3, k21, s50)
sourmash scripts pangenome_merge m_elsdenii.cloud.k21.zip -k 21  \
-o m_elsdenii.cloud.pang.k21.zip --scaled 1000 

sourmash scripts pangenome_ranktable \
m_elsdenii.cloud.pang.k21.zip -o m_elsdenii.cloud.k21.rank.csv \
-k 21 --scaled 1000 

In [ ]:
python ../../../workflow/scripts/calc-hash-presence.py \
m_elsdenii.core.k21.rank.csv pig_sra.txt --scaled=1000 -k 21 -o pig.x.core.dna.dmp


python ../../../workflow/scripts/calc-hash-presence.py \
m_elsdenii.core.k21.rank.csv human_sra.txt --scaled=1000 -k 21 -o human.x.core.dna.dmp

In [ ]:
        python ../../../workflow/scripts/parse-dump.py \
        --dump-files-1 pig.x.core.dna.dmp \
        --dump-files-2 human.x.core.dna.dmp > cmp_core.dna.csv